In [104]:
import pandas as pd
import netCDF4 as nc
import numpy as np
from math import floor
import math
from geopy import distance


In [105]:
grid = pd.DataFrame(columns = ["lat", "long", "population"])

GET GRIDS

In [106]:
import sys
sys.path.append("../tools")
# from grids import pxtolat, pxtolong

In [107]:
def pxtolong(px, width):
    adjlong = (px / width) * 360 
    return adjlong - 180

def pxtolat(px, height):
    adjlat = (px/height) * 180
    return -(adjlat -90)

def closest_solar(lat, lon, arr):
    est_lat = round(lat * 0.05) / 0.05
    est_long = round(lon * 0.05) / 0.05
    
    if est_lat == round(est_lat, 1):
        est_lat -= 0.05

    if est_long == round(est_long, 1):
        est_long -= 0.05

    return arr[(est_lat, est_long)] if (est_lat, est_long) in arr.keys() else None

def closest_winds(lat, lon, arr):
    # 361 576
    # width = 8640
    # height = 4320
    # min_dist = 10 ** 7
    # min_val = None
    # for i in range(len(arr)):
    #     for j in range(len(arr[0])):
    #         dist = distance.distance((lat, lon), (arr[i][j][0], arr[i][j][1])).km
            
    #         if dist < min_dist:
    #             min_dist = dist
    #             min_val = arr[i][j][2]
    est_lat = round(lat * 2) / 2
    est_long = round(lon / 0.625) * 0.625

    if est_lat == 0.0:
        est_lat = 0.5

    if est_long == 180.0:
        est_long = -1.0 * est_long
    if est_long == 0.0:
        est_long = 0.625
    
    return arr[(est_lat, est_long)]


In [108]:
from tqdm import tqdm

d = {
    'lat': [],
    'long': [],
    'population': []
}


with open('../../population_count/gpw_v4_population_count_rev11_2020_2pt5_min.asc', 'r') as f:
        for i in range(6):
            f.readline() # skip pre data
        width = 8640
        height = 4320
        for rn in tqdm(range(0, 4320)):
            k = f.readline().split(' ')
            for c in range(0, 8640):
                cell_val = float(k[c]) # read population in grid

                if cell_val <= 0: # skip pixel if no population
                    continue
                # print(cell_val)
                d['lat'].append(pxtolat(rn, height))
                d['long'].append(pxtolong(c, width))
                d['population'].append(cell_val)
        
        
        grid = pd.DataFrame(d)

100%|██████████| 4320/4320 [00:21<00:00, 201.23it/s]


POPULATION

In [109]:
tmp = grid.loc[grid['population'] > 500000]
tmp.head(10)

,lat,long,population
2742573,45.791667,126.666667,621797.9
2926015,43.916667,125.250000,559883.8
2932979,43.833333,87.583333,520489.2
3108488,41.833333,123.333333,545899.1
3108490,41.833333,123.416667,601812.9
3168988,41.083333,28.833333,629407.6
3168989,41.083333,28.875000,543774.2
3172486,41.041667,28.791667,646545.8
3172487,41.041667,28.833333,818725.8
3172488,41.041667,28.875000,506676.8


EARTHQUAKES

TSUNAMI

HURRICANES

WIND

In [110]:

ds = nc.Dataset('../../energy_strength/wind/avg_wind_speed.nc')
lat = ds['lat'][:]
lon = ds['lon'][:]
speed = ds['M2TMNXFLX_5_12_4_SPEED'][:]



In [111]:
res = {}
for i in tqdm(range(len(lat))):
    for j in range(len(lon)):
        res[(lat[i],lon[j])] = speed[i][j]
res

100%|██████████| 361/361 [00:05<00:00, 68.49it/s]


{(-90.0, -180.0): 8.378205,
 (-90.0, -179.375): 8.378205,
 (-90.0, -178.75): 8.378205,
 (-90.0, -178.125): 8.378205,
 (-90.0, -177.5): 8.378205,
 (-90.0, -176.875): 8.378205,
 (-90.0, -176.25): 8.378205,
 (-90.0, -175.625): 8.378205,
 (-90.0, -175.0): 8.378205,
 (-90.0, -174.375): 8.378205,
 (-90.0, -173.75): 8.378205,
 (-90.0, -173.125): 8.378205,
 (-90.0, -172.5): 8.378205,
 (-90.0, -171.875): 8.378205,
 (-90.0, -171.25): 8.378205,
 (-90.0, -170.625): 8.378205,
 (-90.0, -170.0): 8.378205,
 (-90.0, -169.375): 8.378205,
 (-90.0, -168.75): 8.378205,
 (-90.0, -168.125): 8.378205,
 (-90.0, -167.5): 8.378205,
 (-90.0, -166.875): 8.378205,
 (-90.0, -166.25): 8.378205,
 (-90.0, -165.625): 8.378205,
 (-90.0, -165.0): 8.378205,
 (-90.0, -164.375): 8.378205,
 (-90.0, -163.75): 8.378205,
 (-90.0, -163.125): 8.378205,
 (-90.0, -162.5): 8.378205,
 (-90.0, -161.875): 8.378205,
 (-90.0, -161.25): 8.378205,
 (-90.0, -160.625): 8.378205,
 (-90.0, -160.0): 8.378205,
 (-90.0, -159.375): 8.378205,
 (-90.

In [112]:
tqdm.pandas()
grid['wind_speed'] = grid.progress_apply(lambda x: closest_winds(x['lat'], x['long'], res), axis=1)

100%|██████████| 7550816/7550816 [02:48<00:00, 44837.18it/s]


In [113]:
grid.head(10)

,lat,long,population,wind_speed
0,83.166667,-74.208333,0.000009,4.253750
1,83.166667,-74.166667,0.000011,4.253750
2,83.166667,-74.125000,0.000011,4.253750
3,83.166667,-74.083333,0.000011,4.253750
4,83.166667,-74.041667,0.000008,4.266758
5,83.166667,-74.000000,0.000001,4.266758
6,83.125000,-76.916667,0.000002,4.527839
7,83.125000,-76.875000,0.000001,4.527839
8,83.125000,-74.666667,0.000002,4.253750
9,83.125000,-74.625000,0.000025,4.253750


In [114]:
print(closest_winds(-90.0, -180.0, res))
res
# 8.378205

8.378205


{(-90.0, -180.0): 8.378205,
 (-90.0, -179.375): 8.378205,
 (-90.0, -178.75): 8.378205,
 (-90.0, -178.125): 8.378205,
 (-90.0, -177.5): 8.378205,
 (-90.0, -176.875): 8.378205,
 (-90.0, -176.25): 8.378205,
 (-90.0, -175.625): 8.378205,
 (-90.0, -175.0): 8.378205,
 (-90.0, -174.375): 8.378205,
 (-90.0, -173.75): 8.378205,
 (-90.0, -173.125): 8.378205,
 (-90.0, -172.5): 8.378205,
 (-90.0, -171.875): 8.378205,
 (-90.0, -171.25): 8.378205,
 (-90.0, -170.625): 8.378205,
 (-90.0, -170.0): 8.378205,
 (-90.0, -169.375): 8.378205,
 (-90.0, -168.75): 8.378205,
 (-90.0, -168.125): 8.378205,
 (-90.0, -167.5): 8.378205,
 (-90.0, -166.875): 8.378205,
 (-90.0, -166.25): 8.378205,
 (-90.0, -165.625): 8.378205,
 (-90.0, -165.0): 8.378205,
 (-90.0, -164.375): 8.378205,
 (-90.0, -163.75): 8.378205,
 (-90.0, -163.125): 8.378205,
 (-90.0, -162.5): 8.378205,
 (-90.0, -161.875): 8.378205,
 (-90.0, -161.25): 8.378205,
 (-90.0, -160.625): 8.378205,
 (-90.0, -160.0): 8.378205,
 (-90.0, -159.375): 8.378205,
 (-90.

SOLAR

In [115]:
ds = nc.Dataset('../../energy_strength/solar/avg_solar_energy.nc')
lat = ds['lat'][:]
lon = ds['lon'][:]
energy = ds['FLDAS_NOAH01_CP_GL_M_001_SWdown_f_tavg'][:]

In [116]:
res = {}
for i in tqdm(range(len(lat))):
    for j in range(len(lon)):
        if not math.isnan(energy[i][j]):
            res[(round(lat[i], 2), round(lon[j], 2))] = energy[i][j]


  0%|          | 0/1500 [00:00<?, ?it/s]/Users/julianlioanag/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:4: UserWarning: Warning: converting a masked element to nan.
  after removing the cwd from sys.path.
100%|██████████| 1500/1500 [02:18<00:00, 10.87it/s]


In [117]:
res

{(-55.85, -67.65): 97.825645,
 (-55.85, -67.55): 99.60057,
 (-55.85, -67.35): 101.34623,
 (-55.85, -67.25): 102.19874,
 (-55.85, -67.15): 100.71314,
 (-55.75, -67.45): 98.60866,
 (-55.75, -67.35): 99.10616,
 (-55.75, -67.25): 102.50005,
 (-55.65, -68.05): 96.16191,
 (-55.65, -67.65): 102.52526,
 (-55.65, -67.55): 101.33107,
 (-55.65, -67.45): 101.24142,
 (-55.65, -67.35): 103.76158,
 (-55.55, -68.65): 98.25454,
 (-55.55, -68.55): 101.05223,
 (-55.55, -68.25): 99.65891,
 (-55.55, -68.15): 97.82869,
 (-55.55, -68.05): 99.93344,
 (-55.55, -67.65): 106.09182,
 (-55.45, -69.55): 85.54212,
 (-55.45, -69.45): 87.90538,
 (-55.45, -69.35): 84.338104,
 (-55.45, -69.25): 89.058846,
 (-55.45, -69.15): 98.50018,
 (-55.45, -68.85): 91.66295,
 (-55.45, -68.75): 90.67084,
 (-55.45, -68.65): 88.80512,
 (-55.45, -68.55): 94.37677,
 (-55.45, -68.45): 93.302925,
 (-55.45, -68.35): 97.998085,
 (-55.45, -68.25): 98.51063,
 (-55.45, -68.15): 101.456665,
 (-55.35, -69.95): 85.95961,
 (-55.35, -69.75): 81.7749

In [118]:
ds['FLDAS_NOAH01_CP_GL_M_001_SWdown_f_tavg'][0][0]

masked

In [119]:
grid['solar'] = grid.progress_apply(lambda x: closest_solar(x['lat'], x['long'], res), axis=1)

 88%|████████▊ | 6631810/7550816 [02:40<00:22, 41283.63it/s]


KeyboardInterrupt: 

In [ ]:
grid.loc[grid['solar'] < 100]

,lat,long,population,wind_speed,solar
12640,81.958333,-90.625000,0.000011,6.153218,99.887215
12641,81.958333,-90.583333,0.000027,6.153218,99.887215
12642,81.958333,-90.541667,0.000011,6.153218,99.887215
13288,81.916667,-90.666667,0.000006,6.153218,99.887215
13289,81.916667,-90.625000,0.000007,6.153218,99.887215
...,...,...,...,...,...
1986553,52.875000,-169.041667,0.203065,9.310513,99.148712
1986554,52.875000,-169.000000,0.177392,9.310513,99.148712
1986555,52.875000,-168.958333,0.035516,9.310513,99.148712
1990908,52.833333,-169.125000,0.059100,9.321427,99.148712


In [ ]:
grid.to_csv('grid.csv', index=False)

# MERGING IN CORRUPTION DATA

In [34]:
grid = pd.read_csv('grid.csv')

In [38]:
grid['country'].unique()

array(['CA', 'GL', 'RU', 'SJ', 'NO', 'US', 'FI', 'IS', 'SE', 'FO', 'GB',
       'AX', 'EE', 'LV', 'DK', 'LT', 'BY', 'IE', 'KZ', 'DE', 'PL', 'IM',
       'CN', 'NL', 'MN', 'UA', 'BE', 'FR', 'CZ', 'LU', 'GG', 'SK', 'JE',
       'AT', 'MD', 'HU', 'RO', 'PM', 'CH', 'JP', 'LI', 'IT', 'SI', 'HR',
       'RS', 'UZ', 'BA', 'BG', 'KG', 'SM', 'ES', 'MC', 'GE', 'ME', 'XK',
       'KP', 'AD', 'AL', 'TM', 'MK', 'PT', 'TR', 'VA', 'AZ', 'GR', 'AM',
       'TJ', 'IR', 'AF', 'KR', 'TN', 'IQ', 'PK', 'SY', 'DZ', 'IN', 'GI',
       'MT', 'MA', 'CY', 'LB', 'IL', 'LY', 'MX', 'SA', 'JO', 'PS', 'BM',
       'EG', 'NP', 'KW', 'BT', 'EH', 'BS', 'MM', 'OM', 'BH', 'BD', 'ML',
       'TW', 'QA', 'AE', 'TD', 'MP', 'MR', 'CU', 'VN', 'SD', 'HK', 'MO',
       'NE', 'LA', 'TC', 'TH', 'YE', 'PH', 'HT', 'DO', 'KY', 'VG', 'AI',
       'JM', 'PR', 'BZ', 'VI', 'MF', 'SX', 'BL', 'GT', 'AG', 'BQ', 'SN',
       'HN', 'KN', 'ER', 'CV', 'MS', 'GP', 'BF', 'DM', 'NI', 'MQ', 'ET',
       'SV', 'KH', 'CO', 'GU', 'GM', 'NG', 'LC', 'V

In [36]:
cpi = pd.read_excel("./corruption_index.xlsx", skiprows=2)

In [37]:
cpi['ISO3'].sort_values()
len(cpi['ISO3'].unique())
len(grid['country'].unique())

243

In [27]:
import country_converter as coco

In [39]:
cc = coco.CountryConverter()
iso3_series = cc.pandas_convert(grid['country'], to='ISO3')

nan not found in ISO3


In [40]:
iso3_series.value_counts()

RUS    1693647
CHN     459219
USA     451822
AUS     395858
BRA     369769
        ...   
VAT          6
NFK          6
CCK          4
GIB          1
MCO          1
Name: country, Length: 243, dtype: int64

In [41]:
grid['country'].value_counts()

RU    1693647
CN     459219
US     451822
AU     395858
BR     369769
       ...   
VA          6
NF          6
CC          4
GI          1
MC          1
Name: country, Length: 242, dtype: int64

In [42]:
grid = grid.assign(iso3_name=iso3_series)

In [43]:
import numpy as np
grid.sample(5)

,lat,long,population,wind_speed,solar,country,iso3_name
7149074,-28.083333,118.625000,0.155558,7.278017,269.312592,AU,AUS
3968961,30.958333,54.791667,98.311010,5.994381,224.547180,IR,IRN
2839633,44.791667,0.250000,452.726900,5.971065,208.035019,FR,FRA
4629959,22.375000,16.750000,5.027110,6.506156,277.415070,TD,TCD
3849979,32.333333,47.750000,200.051200,5.802663,231.598450,IR,IRN


It appears that Namibia is being parsed with a country code of Nan. Let's correct that.

In [44]:
grid.loc[grid['iso3_name'].str.contains('not'), 'country'] = "NA"
grid.loc[grid['iso3_name'].str.contains('not'), 'iso3_name'] = "NAM"


In [45]:
grid.rename(columns={'iso3_name': 'iso3_code', 'country': 'iso2_code'}, inplace=True)

Aight now that we've normalized column names and cleaned up the data, we can merge in corruption perception index data based on iso3 country codes.

In [58]:
grid.head()
grid[grid['iso3_code'].isin(['EGY'])]

,lat,long,population,wind_speed,solar,iso2_code,iso3_code
3910180,31.625,30.958333,658.56960,6.013143,231.59845,EG,EGY
3910181,31.625,31.000000,2015.02700,6.013143,231.59845,EG,EGY
3910182,31.625,31.041667,7929.11300,6.013143,231.59845,EG,EGY
3910183,31.625,31.083333,8087.94400,6.013143,231.59845,EG,EGY
3910184,31.625,31.125000,4000.34100,6.013143,231.59845,EG,EGY
...,...,...,...,...,...,...,...
4762318,20.500,32.375000,66.37534,7.541642,NaN,EG,EGY
4762319,20.500,32.416667,66.37534,7.541642,NaN,EG,EGY
4762320,20.500,32.458333,66.37534,7.541642,NaN,EG,EGY
4762321,20.500,32.500000,66.37534,7.541642,NaN,EG,EGY


In [59]:
cpi.columns

Index(['Country / Territory', 'ISO3', 'Region', 'CPI score 2021', 'Rank',
       'Standard error', 'Number of sources', 'Lower CI', 'Upper CI',
       'African Development Bank CPIA',
       'Bertelsmann Foundation Sustainable Governance Index',
       'Bertelsmann Foundation Transformation Index',
       'Economist Intelligence Unit Country Ratings',
       'Freedom House Nations in Transit',
       'Global Insights Country Risk Ratings',
       'IMD World Competitiveness Yearbook', 'PERC Asia Risk Guide',
       'PRS International Country Risk Guide',
       'Varieties of Democracy Project', 'World Bank CPIA',
       'World Economic Forum EOS', 'World Justice Project Rule of Law Index'],
      dtype='object')

In [60]:
merged_grid = grid.merge(cpi[['ISO3', 'CPI score 2021']], left_on='iso3_code', right_on='ISO3')
merged_grid.head()

,lat,long,population,wind_speed,solar,iso2_code,iso3_code,ISO3,CPI score 2021
0,83.166667,-74.208333,0.000009,4.253750,107.335396,CA,CAN,CAN,74
1,83.166667,-74.166667,0.000011,4.253750,107.335396,CA,CAN,CAN,74
2,83.166667,-74.125000,0.000011,4.253750,107.335396,CA,CAN,CAN,74
3,83.166667,-74.083333,0.000011,4.253750,107.335396,CA,CAN,CAN,74
4,83.166667,-74.041667,0.000008,4.266758,107.335396,CA,CAN,CAN,74


In [61]:
merged_grid.drop(columns=["ISO3", "iso2_code"], inplace=True)
merged_grid.rename(columns={"CPI score 2021": "corruption_perception_index"}, inplace=True)

In [62]:
merged_grid.head()

,lat,long,population,wind_speed,solar,iso3_code,corruption_perception_index
0,83.166667,-74.208333,0.000009,4.253750,107.335396,CAN,74
1,83.166667,-74.166667,0.000011,4.253750,107.335396,CAN,74
2,83.166667,-74.125000,0.000011,4.253750,107.335396,CAN,74
3,83.166667,-74.083333,0.000011,4.253750,107.335396,CAN,74
4,83.166667,-74.041667,0.000008,4.266758,107.335396,CAN,74


In [65]:
merged_grid.sample(5)
merged_grid[merged_grid['iso3_code'].isin(['EGY'])]

,lat,long,population,wind_speed,solar,iso3_code,corruption_perception_index
4680833,31.625,30.958333,658.56960,6.013143,231.59845,EGY,33
4680834,31.625,31.000000,2015.02700,6.013143,231.59845,EGY,33
4680835,31.625,31.041667,7929.11300,6.013143,231.59845,EGY,33
4680836,31.625,31.083333,8087.94400,6.013143,231.59845,EGY,33
4680837,31.625,31.125000,4000.34100,6.013143,231.59845,EGY,33
...,...,...,...,...,...,...,...
4696113,20.500,32.375000,66.37534,7.541642,NaN,EGY,33
4696114,20.500,32.416667,66.37534,7.541642,NaN,EGY,33
4696115,20.500,32.458333,66.37534,7.541642,NaN,EGY,33
4696116,20.500,32.500000,66.37534,7.541642,NaN,EGY,33


In [66]:
merged_grid.to_csv('./merged_cpi_dataset.csv', index=False)

# Conflict merge

In [67]:
grid = pd.read_csv('./merged_cpi_dataset.csv')

In [68]:
grid.sample(5)

,lat,long,population,wind_speed,solar,iso3_code,corruption_perception_index
7275131,-28.708333,23.500000,4.047177,6.546346,253.720901,ZAF,44
5218688,17.916667,-77.416667,4809.250000,6.243315,NaN,JAM,44
6983208,-31.791667,143.166667,0.146677,6.468293,NaN,AUS,73
6450166,-7.916667,39.625000,3054.606000,5.810673,259.100891,TZA,39
7446225,-43.125000,-66.125000,17.806460,8.242099,NaN,ARG,38


In [69]:
con = pd.read_csv('../../conflicts/african_conflicts.csv')

In [70]:
con['Target Country'].unique()
len(con['Target Country'].unique())

51

In [71]:
import country_converter as coco
cc = coco.CountryConverter()
iso3_series = cc.pandas_convert(con['Target Country'], to='ISO3')

In [72]:
con = con.assign(iso3=iso3_series)
con[['Intensity', 'Target Country', 'iso3']]

,Intensity,Target Country,iso3
0,-7.2,Benin,BEN
1,3.4,Egypt,EGY
2,0.4,Egypt,EGY
3,0.0,Nigeria,NGA
4,0.0,Sudan,SDN
...,...,...,...
260710,-4.0,Senegal,SEN
260711,3.4,Democratic Republic of Congo,COD
260712,3.0,Cote d'Ivoire,CIV
260713,3.0,Cote d'Ivoire,CIV


In [73]:
con = con[con['Intensity'] <= -5]
agg_functions = {'Intensity': 'count', 'iso3': 'first'}
con_agg = con.groupby(con['iso3'], as_index=False).aggregate(agg_functions)

In [76]:
con_agg['iso3'].unique()
grid[grid['iso3_code'].isin(['EGY'])]

,lat,long,population,wind_speed,solar,iso3_code,corruption_perception_index
4680833,31.625,30.958333,658.56960,6.013143,231.59845,EGY,33
4680834,31.625,31.000000,2015.02700,6.013143,231.59845,EGY,33
4680835,31.625,31.041667,7929.11300,6.013143,231.59845,EGY,33
4680836,31.625,31.083333,8087.94400,6.013143,231.59845,EGY,33
4680837,31.625,31.125000,4000.34100,6.013143,231.59845,EGY,33
...,...,...,...,...,...,...,...
4696113,20.500,32.375000,66.37534,7.541642,NaN,EGY,33
4696114,20.500,32.416667,66.37534,7.541642,NaN,EGY,33
4696115,20.500,32.458333,66.37534,7.541642,NaN,EGY,33
4696116,20.500,32.500000,66.37534,7.541642,NaN,EGY,33


In [77]:
merged_grid = grid.merge(con_agg[['Intensity', 'iso3']], left_on='iso3_code', right_on='iso3')

In [78]:
merged_grid.rename(columns={'Intensity': 'conflict_intensity'}, inplace=True)
merged_grid.drop('iso3', axis=1, inplace=True)

In [79]:
# len(merged_grid)
# len(con)
# len(grid)
merged_grid.sample(5)

,lat,long,population,wind_speed,solar,iso3_code,corruption_perception_index,conflict_intensity
503439,13.291667,33.208333,1395.63500,6.321477,NaN,SDN,20,5454
656570,12.833333,13.958333,1127.37000,6.329451,277.415070,NGA,24,14633
157732,30.708333,24.708333,44.81238,6.216860,202.255997,LBY,17,2245
992722,-9.708333,24.958333,721.76690,2.755471,219.564026,COD,19,1774
1340816,-23.875000,32.166667,186.42600,5.943557,NaN,ZAF,44,4130


In [86]:
merged_grid[merged_grid['iso3_code'].isin(['EGY'])]
len(merged_grid['iso3_code'].unique())

51

In [83]:
merged_grid.to_csv('./africa_merged_dataset.csv', index=False)